In [ ]:
!conda install -c conda-forge xgboost --yes

In [29]:
!conda install -c conda-forge category_encoder

Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - category_encoder

Current channels:

  - https://conda.anaconda.org/conda-forge/osx-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/free/osx-64
  - https://repo.anaconda.com/pkgs/free/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [151]:
import pandas as pd 
import numpy as np 
import seaborn as sns 

In [242]:
test_features = pd.read_csv('test_features.csv')
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

In [243]:
pd.set_option('display.max_columns', 1000)
test_features.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [244]:
train_labels.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [245]:
test_features.shape, train_features.shape, train_labels.shape

((14358, 40), (59400, 40), (59400, 2))

In [246]:
#majority class baseline accuracy score ~ 54% 
#our model should perform better then our naive basemodel 
train_labels['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [247]:
#since this is a classification problem, we will be using tree based models such as XGBoost and Random Forest
#overfitting could be an issue here, as 50% of the test set is held out in the private leaderboard 
#we will subdivide our training data set onto training and validation sets, we will use cross-validation 
#with independent test set, however we will be testing first on our new Kaggle subdivided validation set 

In [248]:
from sklearn.model_selection import train_test_split
X_train, X_train_val, y_train, y_train_val  = train_test_split(train_features, train_labels, test_size=0.2)

In [249]:
 X_train.shape, X_train_val.shape, y_train.shape, y_train_val.shape

((47520, 40), (11880, 40), (47520, 2), (11880, 2))

In [250]:
X_train.columns.tolist()

    
features = ['installer',
'construction_year',   
'amount_tsh',           
'waterpoint_type',      
'quantity']   


In [279]:
#X_train.columns.tolist()

In [266]:
X_train = X_train[features]
y_train = y_train[['status_group']]

In [267]:
X_train.head()

,installer,construction_year,amount_tsh,waterpoint_type,quantity
17915,DWE,1972,0.0,other,enough
54082,MAKE ENGINEERING,0,0.0,hand pump,insufficient
37669,DWE,1989,0.0,other,enough
378,KIUMA,2011,50.0,communal standpipe multiple,enough
12077,RWE,1996,500.0,hand pump,insufficient


In [268]:
y_train.head()

,status_group
17915,non functional
54082,non functional
37669,non functional
378,functional needs repair
12077,functional


In [269]:
import category_encoders as ce
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

In [270]:
encoder = ce.OrdinalEncoder()

In [280]:
encoder = ce.OrdinalEncoder()
X_train = encoder.fit_transform(X_train)

param_distributions = {
    'n_estimators': [100, 200]
    'max_depth': [2, 3]
}

# n_iter & cv parameters are low here so the example runs faster
search = RandomizedSearchCV(
    estimator=XGBClassifier(n_jobs=-1, random_state=42), 
    param_distributions=param_distributions, 
    n_iter=3, 
    #scoring=, 
    n_jobs=-1, 
    cv=2, 
    verbose=10, 
    return_train_score=True, 
    random_state=42
)

search.fit(X_train, y_train)

SyntaxError: invalid syntax (<ipython-input-280-77650fca8f34>, line 6)

In [ ]:
best = search.best_estimator_
#pd.Series(best.feature_importances_, X_train.columns).sort_values().tail().plot.barh(color='gray')

In [ ]:
search.best_score_

In [ ]:
best = search.best_estimator_
best

In [ ]:
#pd.DataFrame(best.predict(X_test))

In [ ]:
#X_test = X_test.rename(columns={'installer': 'f2', 'construction_year':'f1', 'amount_tsh':'f4',
#                               'waterpoint_type': 'f0', 'quantity':'f3'})
#X_test.head()

In [271]:
X_test = test_features[features]
X_test = encoder.fit_transform(X_test)

In [274]:
X_test.head()

,installer,construction_year,amount_tsh,waterpoint_type,quantity
0,1,2012,0.0,1,1
1,2,2000,0.0,2,2
2,3,2010,0.0,1,2
3,4,1987,0.0,1,3
4,5,2000,500.0,2,4


In [ ]:
y_pred = pd.DataFrame(best.predict(X_test))
submit_predictions = y_pred
sub = pd.DataFrame(submit_predictions)
sub.head()

In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('test4.csv', index=False)

In [ ]:
submission.head(20)
#submission = submission.replace({1: 'functional', 2: 'non functional'})
#submission.head()

In [272]:
pred = pd.DataFrame(best.predict(X_test))
pred.head()

,0
0,non functional
1,functional
2,functional
3,non functional
4,non functional


In [ ]:
#submission.to_csv('sub_test.csv', index=False)

In [140]:
X_train_val = X_train_val[features]
X_train_val = encoder.fit_transform(X_train_val)

In [265]:
#from sklearn.metrics import accuracy_score#
#y_pred_proba = best.predict_proba(X_train_val)[:,0]
#print('Accuracy', accuracy_score(y_train_val, y_pred_proba))

In [149]:
pd.DataFrame(y_pred_proba).head()

,0
0,0.631584
1,0.432892
2,0.392198
3,0.365617
4,0.361863


In [275]:
y_train.head()

,status_group
17915,non functional
54082,non functional
37669,non functional
378,functional needs repair
12077,functional


In [281]:
#encoder.fit_transform(y_train)

In [262]:
y_train.head()

,status_group
17915,non functional
54082,non functional
37669,non functional
378,functional needs repair
12077,functional


In [260]:
from sklearn.ensemble import RandomForestClassifier

param_distributions = {
    'n_estimators': [100, 200], 
    'max_depth': [4, 5], 
}

gridsearch = RandomizedSearchCV(
    RandomForestClassifier(n_jobs=-1, random_state=42), 
    param_distributions=param_distributions, 
    n_iter=8, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/Users/TomasFox/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 4 is smaller than n_iter=8. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  11 | elapsed:    7.4s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done   8 out of  11 | elapsed:    7.4s remaining:    2.8s


ValueError: could not convert string to float: 'DWE'